In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as datetime
from collections import Counter
import time
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report

db = pd.read_csv('caltech_clean.csv')
db.head()

,Unnamed: 0,connectionTime,disconnectTime,siteID,spaceID,userID,userInputs/0/WhPerMile,userInputs/0/kWhRequested,userInputs/0/milesRequested,userInputs/0/minutesAvailable,userInputs/0/modifiedAt,userInputs/0/paymentRequired,userInputs/0/requestedDeparture
0,0,7.187222,15.451667,2,315,743.0,400.0,8.00,20.0,30.0,7.191944,1.0,7.687222
1,1,7.718056,23.609722,2,316,4269.0,400.0,16.00,40.0,60.0,7.865000,1.0,8.718056
2,2,7.867222,20.912778,2,303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,8.414444,19.320833,2,323,3853.0,360.0,21.60,60.0,559.0,8.418889,1.0,17.731111
4,4,8.984722,23.126944,2,492,1470.0,292.0,20.44,70.0,400.0,8.990556,1.0,15.651389


In [8]:
X = db[["connectionTime", "siteID", "spaceID"]].values
X

y = db['disconnectTime'].values
#label encoding is done as model accepts only numeric values
# so strings need to be converted into labels
LE = preprocessing.LabelEncoder()
LE.fit(y)
y = LE.transform(y)
y

#splitting dataset into train, validation and test data
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state = 0)
X_train,X_val,y_train,y_val = train_test_split(X_train,y_train,test_size=0.25,random_state = 0)

# Scale data
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Train Logistic Regression model
train_logreg = LogisticRegression(random_state=0, max_iter=200).fit(X_train, y_train)

# Train a Neural Network (MLPClassifier)
train_nn = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam', max_iter=500, random_state=0)
train_nn.fit(X_train, y_train)

# Predictions for Logistic Regression
pred_logreg = train_logreg.predict(X_val)
print("For Logistic Regression: ")
print(classification_report(y_val, pred_logreg))
print("Accuracy of logistic regression on the initial data is: ", accuracy_score(y_val, pred_logreg))

# Predictions for Neural Network
pred_nn = train_nn.predict(X_val)
print("For Neural Network: ")
print(classification_report(y_val, pred_nn))
print("Accuracy of NN on the initial data is: ", accuracy_score(y_val, pred_nn))

For Logistic Regression: 
              precision    recall  f1-score   support

           2       0.00      0.00      0.00       0.0
           4       0.00      0.00      0.00       1.0
          23       0.00      0.00      0.00       1.0
          25       0.00      0.00      0.00       1.0
          27       0.00      0.00      0.00       1.0
          28       0.00      0.00      0.00       1.0
          37       0.00      0.00      0.00       1.0
          38       0.00      0.00      0.00       1.0
          41       0.00      0.00      0.00       1.0
          45       0.00      0.00      0.00       1.0
          52       0.00      0.00      0.00       1.0
          57       0.00      0.00      0.00       1.0
          59       0.00      0.00      0.00       1.0
          65       0.00      0.00      0.00       1.0
          75       0.00      0.00      0.00       1.0
          80       0.00      0.00      0.00       1.0
          81       0.00      0.00      0.00       1.0
 

c:\Users\chris\anaconda3\envs\ME44312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\chris\anaconda3\envs\ME44312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\chris\anaconda3\envs\ME44312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\chris\anaconda3\envs\ME44312\Lib\site-packages\sklearn\metrics\_classification.py:1565: 